In [2]:
import sqlite3
from pandas.io import sql
import subprocess
%load_ext sql

In [3]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
import pandas as pd
# Load environment variables from the .env file
load_dotenv()

# Retrieve the database credentials from environment variables
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")

# Use the credentials in your database connection code
connection_string = f"postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
# ... rest of your code ...
engine = create_engine(connection_string)


# Revenue/country

In [7]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}

SELECT SUM(USERS_PAY_COUNT) AS SUM_PAY,
       COUNTRY
FROM
        (SELECT COUNT(*) AS USERS_PAY_COUNT,
        R.USER_ID,
        R.COUNTRY,
        B.PRICE
    FROM READ_DATA AS R
    JOIN BUY_DATA AS B ON R.USER_ID = B.USER_ID
    GROUP BY R.USER_ID,
        B.PRICE,
        R.COUNTRY
    ORDER BY R.COUNTRY) AS MAIN_QUERY
GROUP BY COUNTRY
ORDER BY SUM_PAY DESC;

8 rows affected.


sum_pay,country
47808,country_5
34036,country_2
33654,country_7
26998,country_4
13370,country_6
3002,country_3
2134,country_8
585,country_1


# Monthly readers/subscribers/buyers

In [8]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT 
    DATE_TRUNC('month', my_date) AS month,
    COUNT(*) AS buyers
FROM buy_data
GROUP BY month
ORDER BY month;

3 rows affected.


month,buyers
2018-01-01 00:00:00+00:00,947
2018-02-01 00:00:00+00:00,2564
2018-03-01 00:00:00+00:00,4895


In [10]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT 
    DATE_TRUNC('month', my_date) AS month,
    COUNT(*) AS readers
FROM read_data
GROUP BY month
ORDER BY month;

3 rows affected.


month,readers
2018-01-01 00:00:00+00:00,99932
2018-02-01 00:00:00+00:00,189657
2018-03-01 00:00:00+00:00,292287


In [11]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT 
    DATE_TRUNC('month', my_date) AS month,
    COUNT(*) AS subscribers
FROM subscribe_data
GROUP BY month
ORDER BY month;

3 rows affected.


month,subscribers
2018-01-01 00:00:00+00:00,2043
2018-02-01 00:00:00+00:00,2450
2018-03-01 00:00:00+00:00,3124


# Funnel analysis breakdown for all users by date

In [12]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT
    r.my_date,
    COUNT(DISTINCT CASE WHEN r.user_id IS NOT NULL THEN r.user_id ELSE NULL END) AS readers,
    COUNT(DISTINCT CASE WHEN s.user_id IS NOT NULL THEN r.user_id ELSE NULL END) AS subscribers,
    COUNT(DISTINCT CASE WHEN b.user_id IS NOT NULL THEN r.user_id ELSE NULL END) AS buyers
FROM
    read_data AS r
    LEFT JOIN subscribe_data AS s ON r.user_id = s.user_id
    LEFT JOIN buy_data AS b ON r.user_id = b.user_id
GROUP BY
    r.my_date
ORDER BY
    r.my_date;

90 rows affected.


my_date,readers,subscribers,buyers
2018-01-01,1794,280,83
2018-01-02,1849,269,91
2018-01-03,1886,233,86
2018-01-04,1987,267,130
2018-01-05,2043,266,141
2018-01-06,2102,305,148
2018-01-07,2107,300,146
2018-01-08,2161,337,169
2018-01-09,2271,383,185
2018-01-10,2288,387,188


# All users on each day

In [ ]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT r.my_date,
    r.my_time,
    r.user_id,
    COUNT(DISTINCT(r.user_id)) AS readers,
    COUNT(DISTINCT(s.user_id)) AS subscribers,
    COUNT(DISTINCT(b.user_id)) AS buyers,
    r.topic,
    b.price,
    r.source
FROM read_data AS r
LEFT JOIN subscribe_data AS s ON r.user_id = s.user_id
LEFT JOIN buy_data AS b ON r.user_id = b.user_id
GROUP BY r.my_date,
    r.user_id,
    r.my_time,
    r.topic,
    b.price,
    r.source;

624804 rows affected.


# Daily readers

In [5]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT my_date, SUM(user_count) AS daily_readers
FROM
(SELECT my_date, user_id, COUNT(*) as user_count
FROM read_data
GROUP BY my_date, user_id) AS main
GROUP BY my_date;

90 rows affected.


my_date,daily_readers
2018-01-01,1866
2018-01-02,1896
2018-01-03,2013
2018-01-04,2276
2018-01-05,2285
2018-01-06,2398
2018-01-07,2379
2018-01-08,2505
2018-01-09,2896
2018-01-10,2999


# Daily subscribers

In [6]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT my_date, SUM(user_count) AS daily_subscribers
FROM
(SELECT my_date, user_id, COUNT(*) AS user_count
FROM subscribe_data
GROUP BY my_date, user_id) AS main
GROUP BY my_date
ORDER BY my_date;

89 rows affected.


my_date,daily_subscribers
2018-01-01,59
2018-01-02,115
2018-01-03,181
2018-01-04,47
2018-01-05,42
2018-01-06,36
2018-01-07,56
2018-01-08,38
2018-01-09,50
2018-01-10,55


# Daily buyers

In [7]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT my_date, SUM(user_count) AS daily_buyers
FROM
(SELECT my_date, user_id, COUNT(*) AS user_count
FROM buy_data
GROUP BY my_date, user_id) as main
GROUP BY my_date
ORDER BY my_date;

89 rows affected.


my_date,daily_buyers
2018-01-01,6
2018-01-02,11
2018-01-03,11
2018-01-04,12
2018-01-05,15
2018-01-06,19
2018-01-07,17
2018-01-08,26
2018-01-09,9
2018-01-10,22


# Users/country

In [8]:
%%sql postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}
SELECT country, COUNT(*)
FROM read_data
GROUP BY country;

8 rows affected.


country,count
country_1,6231
country_2,130076
country_3,9128
country_4,71258
country_5,149732
country_6,63752
country_7,132066
country_8,19633
